In [12]:


import pyspark as ps
import warnings
from pyspark.sql import SQLContext


from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, LinearSVC, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline



In [13]:

sc = ps.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)


In [14]:
df_train = sqlContext.read.format('com.databricks.spark.csv').options(header = False, inferschema = True, sep = ";").load("data/train.csv")
df_train= df_train.withColumnRenamed('_c0','description').withColumnRenamed('_c1','feeling')
df_train.show(10)

+--------------------+--------+
|         description| feeling|
+--------------------+--------+
|i didnt feel humi...| sadness|
|i can go from fee...| sadness|
|im grabbing a min...|   anger|
|i am ever feeling...|    love|
|i am feeling grouchy|   anger|
|ive been feeling ...| sadness|
|ive been taking o...|surprise|
|i feel as confuse...|    fear|
|i have been with ...|     joy|
| i feel romantic too|    love|
+--------------------+--------+
only showing top 10 rows



In [15]:
df_test = sqlContext.read.format('com.databricks.spark.csv').options(header = False, inferschema = True, sep = ";").load("data/val.csv")
df_test= df_test.withColumnRenamed('_c0','description').withColumnRenamed('_c1','feeling')
df_test.show(10)

+--------------------+-------+
|         description|feeling|
+--------------------+-------+
|im feeling quite ...|sadness|
|i feel like i am ...|sadness|
|i feel like a fai...|   love|
|i am just feeling...|  anger|
|i can have for a ...|    joy|
|i start to feel m...|    joy|
|i am feeling more...|    joy|
|i feel incredibly...|    joy|
|i feel less keen ...|    joy|
|i feel dirty and ...|sadness|
+--------------------+-------+
only showing top 10 rows



In [16]:
print(df_train.count())
df_train.dropna()
df_train.count()

16000


16000

In [17]:
print(df_test.count())
df_test.dropna()
df_test.count()

2000


2000

In [18]:
label_stringIdx = StringIndexer(inputCol = 'feeling', outputCol = 'label')
tokenizer = Tokenizer(inputCol = 'description', outputCol = 'tokens')
hashingtf = HashingTF(numFeatures= 2**16, inputCol = "tokens", outputCol = 'tf')
idf = IDF(inputCol = "tf", outputCol = 'features', minDocFreq=3)

In [19]:
pipeline = Pipeline(stages = [label_stringIdx, tokenizer, hashingtf, idf ])

In [20]:
pipeline = pipeline.fit(df_train)
df_train = pipeline.transform(df_train)
df_test = pipeline.transform(df_test)

In [21]:
df_train.show(10)

+--------------------+--------+-----+--------------------+--------------------+--------------------+
|         description| feeling|label|              tokens|                  tf|            features|
+--------------------+--------+-----+--------------------+--------------------+--------------------+
|i didnt feel humi...| sadness|  1.0|[i, didnt, feel, ...|(65536,[13351,244...|(65536,[13351,244...|
|i can go from fee...| sadness|  1.0|[i, can, go, from...|(65536,[3266,8436...|(65536,[3266,8436...|
|im grabbing a min...|   anger|  2.0|[im, grabbing, a,...|(65536,[8436,2441...|(65536,[8436,2441...|
|i am ever feeling...|    love|  4.0|[i, am, ever, fee...|(65536,[3266,7194...|(65536,[3266,7194...|
|i am feeling grouchy|   anger|  2.0|[i, am, feeling, ...|(65536,[2187,3266...|(65536,[2187,3266...|
|ive been feeling ...| sadness|  1.0|[ive, been, feeli...|(65536,[3053,3266...|(65536,[3053,3266...|
|ive been taking o...|surprise|  5.0|[ive, been, takin...|(65536,[6834,1045...|(65536,[6834

In [22]:
df_test.show(10)

+--------------------+-------+-----+--------------------+--------------------+--------------------+
|         description|feeling|label|              tokens|                  tf|            features|
+--------------------+-------+-----+--------------------+--------------------+--------------------+
|im feeling quite ...|sadness|  1.0|[im, feeling, qui...|(65536,[3266,9318...|(65536,[3266,9318...|
|i feel like i am ...|sadness|  1.0|[i, feel, like, i...|(65536,[731,8315,...|(65536,[731,8315,...|
|i feel like a fai...|   love|  4.0|[i, feel, like, a...|(65536,[10172,116...|(65536,[10172,116...|
|i am just feeling...|  anger|  2.0|[i, am, just, fee...|(65536,[3266,2441...|(65536,[3266,2441...|
|i can have for a ...|    joy|  0.0|[i, can, have, fo...|(65536,[2089,3266...|(65536,[2089,3266...|
|i start to feel m...|    joy|  0.0|[i, start, to, fe...|(65536,[3024,8436...|(65536,[3024,8436...|
|i am feeling more...|    joy|  0.0|[i, am, feeling, ...|(65536,[3266,8436...|(65536,[3266,8436...|


In [23]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [24]:
nb = nb.fit(df_train)

In [25]:
predictions = nb.transform(df_test)

In [26]:
criterion = MulticlassClassificationEvaluator()
criterion.evaluate(predictions)

2022-01-26 20:36:50 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2022-01-26 20:36:50 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


0.7375165137942361